In [1]:
import os
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


In [2]:
# Define dataset path
DATASET_PATH = "D:/Music_v3/Emotion_Based_Music_Player_with_Age_Preference/UTKFace"

In [3]:
# Define categories (Modify if needed)
AGE_CATEGORIES = {
    "Child": (0, 12),
    "Teenager": (13, 19),
    "Adult": (20, 59),
    "Senior": (60, 100),
}


In [4]:
# Load images and labels
IMG_SIZE = 64  # Resize images to 64x64
X, Y = [], []


In [13]:
for img_name in os.listdir(DATASET_PATH):
    try:
        # Extract age from filename (e.g., "25_0_0_20170116174525125.jpg")
        age = int(img_name.split("_")[0])

        # Assign category
        label = None
        for category, (min_age, max_age) in AGE_CATEGORIES.items():
            if min_age <= age <= max_age:
                label = category
                break

        if label:
            img_path = os.path.join(DATASET_PATH, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = img / 255.0  # Normalize

            X.append(img)
            Y.append(label)
    except Exception as e:
        print(f"Skipping {img_name}: {e}")

In [16]:
# Convert to NumPy arrays
X = np.array(X)
Y = np.array(Y)

# Convert labels to categorical (One-hot encoding)
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y)
Y_onehot = to_categorical(Y_encoded)

# Train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_onehot, test_size=0.2, random_state=42)


In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(len(AGE_CATEGORIES), activation="softmax")  # Output layer
])

# Compile Model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Summary
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 683,716 (2.61 MB)

 Trainable params: 683,716 (2.61 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
from tensorflow.keras.callbacks import EarlyStopping

# Define Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train Model with Early Stopping
EPOCHS = 20
BATCH_SIZE = 32

history = model.fit(X_train, Y_train,
                    validation_data=(X_test, Y_test),
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    callbacks=[early_stopping])  # Add Early Stopping callback


Epoch 1/20
603/603 ━━━━━━━━━━━━━━━━━━━━ 33s 50ms/step - accuracy: 0.6768 - loss: 0.9395 - val_accuracy: 0.7150 - val_loss: 0.7854
Epoch 2/20
603/603 ━━━━━━━━━━━━━━━━━━━━ 29s 49ms/step - accuracy: 0.7088 - loss: 0.8086 - val_accuracy: 0.7258 - val_loss: 0.7563
Epoch 3/20
603/603 ━━━━━━━━━━━━━━━━━━━━ 28s 47ms/step - accuracy: 0.7310 - loss: 0.7404 - val_accuracy: 0.7362 - val_loss: 0.7146
Epoch 4/20
603/603 ━━━━━━━━━━━━━━━━━━━━ 28s 46ms/step - accuracy: 0.7594 - loss: 0.6802 - val_accuracy: 0.7680 - val_loss: 0.6610
Epoch 5/20
603/603 ━━━━━━━━━━━━━━━━━━━━ 28s 46ms/step - accuracy: 0.7802 - loss: 0.6261 - val_accuracy: 0.7796 - val_loss: 0.6502
Epoch 6/20
603/603 ━━━━━━━━━━━━━━━━━━━━ 28s 47ms/step - accuracy: 0.7868 - loss: 0.5827 - val_accuracy: 0.7827 - val_loss: 0.6195
Epoch 7/20
603/603 ━━━━━━━━━━━━━━━━━━━━ 28s 47ms/step - accuracy: 0.8032 - loss: 0.5272 - val_accuracy: 0.7895 - val_loss: 0.6374
Epoch 8/20
603/603 ━━━━━━━━━━━━━━━━━━━━ 28s 47ms/step - accuracy: 0.8226 - loss: 0.4792 - 

In [22]:
model.save("age_classifier_model.h5")
print("Model saved successfully!")

Model saved successfully!


In [24]:
def predict_age_category(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    prediction = model.predict(img)
    predicted_label = label_encoder.inverse_transform([np.argmax(prediction)])

    return predicted_label[0]

# Test
test_image = "D:/portofolio/portfolio/66b4db66-1ee5-49e5-bfb4-f372efbebc00_jf01nkk.jpeg"
print("Predicted Age Category:", predict_age_category(test_image))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
Predicted Age Category: Adult


In [25]:
print(model.input_shape)


(None, 64, 64, 3)
